In [43]:
# import openpyxl module 
import openpyxl 
import re
  
# Give the location of the file 
path = "/Users/mokrzesik/Desktop/Michael/General/Disney/05-01-2024 Disney Dining.xlsx"

# set the sheet names of the workbook
sheet1 = 'Restaurant List'
sheet2 = 'Menus'
  
# To open the workbook 
# workbook object is created 
wb = openpyxl.load_workbook(path) 
  
# Get menu workbook
# from the active attribute 
menu = wb.worksheets[1]

In [269]:
decimal_pattern = re.compile(r'[-+]?\d*\.\d{2}')
dollar_sign = re.compile(r'\$')
market_price = re.compile(r'market price', re.IGNORECASE)
number_pattern = re.compile(r'\d+')
date_pattern = re.compile(r"['’‘]\d+")
age_pattern = re.compile(r'\d+[+]')
year_pattern = re.compile(r'\d{4}')
ounce_pattern = re.compile(r'oz')
liquid_pattern = re.compile(r'mL|L')
doubledigit_pattern = re.compile(r'\d{2}')
multiple_pattern = re.compile(r'\d+\s*\|\s*\d+\s*\|\s*\d+')
singledigit_pattern = re.compile(r'\d{1}')


def value_check(string):
  # checks if there is a dollar sign or has decimal pattern
  if dollar_sign.search(string) or decimal_pattern.search(string) or multiple_pattern.search(string):
    return True
  elif (date_pattern.search(string) or age_pattern.search(string) or year_pattern.search(string) or ounce_pattern.search(string) 
        or liquid_pattern.search(string) or doubledigit_pattern.search(string) or singledigit_pattern.search(string)):
    return False
  elif number_pattern.search(string):
    return True
  else:
    return False
  
def process_string(input_string):
  # Remove dollar signs
  string = re.sub(r'\$', '', input_string)
  string = re.sub(r'beverages?', '', string, flags=re.IGNORECASE)
  string = re.sub(r'child\s*\(?(\d+-)?\d\)?', '', string, flags=re.IGNORECASE)
  string = re.sub(r'ages\s*\(?(\d+-)?\d\)?', '', string, flags=re.IGNORECASE)
  string = re.sub(r'[a-zA-Z]+\s*\d{4}\s*[a-zA-Z]+', '', string)

  # Extract numbers with hyphen if necessary
  numbers = re.findall(r'\d+(?:\.\d+)?-?\d+(?:\.\d+)?', string)

  # Join the numbers into a single string
  result = '-'.join(numbers)
  
  return result

In [270]:
for i, (cellA, cellD) in enumerate(zip(menu['A'], menu['D'])):
  if menu[f'B{i+1}'].value is not None:
    continue
  if value_check(cellA.value):
    # print(cellA.value)
    result = process_string(cellA.value)
    print(f'Result from A row_{i+1}: {result}')
  if value_check(cellD.value):
    # print(cellD.value)
    result = process_string(cellD.value)
    print(f'Result from D row_{i+1}: {result}')

Result from D row_45: 8.75-12.25
Result from D row_46: 8.75-12.25
Result from D row_47: 8.75-12.25
Result from D row_48: 8.75-12.25
Result from D row_49: 8.75-12.25
Result from D row_50: 8.75-12.25
Result from A row_361: 5.39-5.99
Result from D row_381: 8.75-12.25
Result from D row_382: 8.75-12.25
Result from D row_383: 8.75-12.25
Result from D row_384: 8.75-12.25
Result from D row_385: 8.75-12.25
Result from D row_386: 8.75-12.25
Result from D row_387: 8.75-12.25
Result from D row_388: 8.75-12.25
Result from D row_389: 8.75-12.25
Result from D row_390: 8.75-12.25
Result from D row_391: 8.75-12.25
Result from A row_420: 8.50-12.75
Result from A row_421: 11.00-16.00
Result from A row_490: 54
Result from A row_491: 34
Result from A row_529: 66
Result from A row_530: 41
Result from A row_1642: 1.79-7.49
Result from D row_1686: 8.75-12.25
Result from D row_1687: 8.75-12.25
Result from D row_1688: 8.75-12.25
Result from D row_1689: 8.75-12.25
Result from D row_1690: 8.75-12.25
Result from D

In [265]:
print(process_string('Assorted Bottled Beverages ($4.49 -$4.69)'))

49-4.69
